In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
encoder_input = tf.keras.Input(shape=(2))
x = layers.Dense(3, activation="relu", name="layer1")(encoder_input)
#x = layers.Dense(1, activation="relu")(x)
encoder_output = layers.Dense(1, name="end_layer")(x)

model = tf.keras.Model(encoder_input, encoder_output, name="encoder")

model.summary()

In [ ]:
print(model.get_layer("layer1").get_weights())

a = np.array([[1,0,0],[-1,1,-1]])
b = np.array([0,0,0])
model.get_layer("layer1").set_weights([a,b])
print(model.get_layer("layer1").get_weights())

In [ ]:
print(model.get_layer("end_layer").get_weights())

a = np.array([1,1,-1]).reshape((3,1))
b = np.array([0])

model.get_layer("end_layer").set_weights([a,b])
print(model.get_layer("end_layer").get_weights())



In [ ]:
tensor = tf.convert_to_tensor(np.array([3,2]).reshape(1,2))
print(float(model(tensor)))

In [ ]:
def create_max_model(name1,name2):

    max_input = tf.keras.Input(shape=(2))
    x = layers.Dense(3, activation="relu", name="layer{}".format(name1))(max_input)
    max_output = layers.Dense(1, name="layer{}".format(name2))(x)
    max_model = tf.keras.Model(max_input, max_output, name="max")
    a = np.array([[1,0,0],[-1,1,-1]])
    b = np.array([0,0,0])
    max_model.get_layer("layer{}".format(name1)).set_weights([a,b])
    a = np.array([1,1,-1]).reshape((3,1))
    b = np.array([0])
    max_model.get_layer("layer{}".format(name2)).set_weights([a,b])
    return max_model#, max_input, max_output

a_model= create_max_model("a1","a2")
b_model = create_max_model("b1","b2")

#print(a_model(tensor))
#m_model.summary()

In [ ]:
#TODO: define function that generates a model
#TODO: chain them


#encoder_input = tf.keras.Input(shape=(2))
x = layers.Concatenate()([a_model.output, b_model.output])
#x = layers.Concatenate()([a_model.output, layers.Input(1)])
#x = layers.Reshape((2,), name="layer_11")(x)
x = layers.Dense(3, activation="relu", name="layer1")(x)
larger_max = layers.Dense(1, name="layer_21")(x)

model_b = tf.keras.Model([a_model.input, b_model.input], larger_max, name="encoder")
#model_b = tf.keras.Model([a_model.input,  layers.Input(1)], larger_max, name="encoder")

a = np.array([[1,0,0],[-1,1,-1]])
b = np.array([0,0,0])
model_b.layers[-2].set_weights([a,b])

a = np.array([1,1,-1]).reshape((3,1))
b = np.array([0])
model_b.layers[-1].set_weights([a,b])


model_b.summary()
a1 = tf.convert_to_tensor( np.array([1,2]).reshape(1,2))
b1 = tf.convert_to_tensor( np.array([1,2]).reshape(1,2))

print(a_model(b1))
print(model_b([a1,b1]))
#tf.keras.utils.plot_model(model_b)

In [ ]:
a_test = tf.convert_to_tensor( np.array([2.3]).reshape(1,1))
newIn1 = layers.Input(batch_shape = (1,1))
newOut1 = layers.Dense(1,activation=lambda xy: xy, name="a")(newIn1)
new_model = tf.keras.Model(newIn1, newOut1)
a = np.array([1]).reshape((1,1))
b = np.array([0])
new_model.get_layer("a").set_weights([a,b])
new_model.summary()

new_model(a_test)

In [ ]:

def create_max_model_rev():

    max_input = tf.keras.Input(shape=(2))
    x = layers.Dense(3, activation="relu")(max_input)
    max_output = layers.Dense(1)(x)
    max_model = tf.keras.Model(max_input, max_output, name="max")
    a = np.array([[1,0,0],[-1,1,-1]])
    b = np.array([0,0,0])
    max_model.layers[1].set_weights([a,b])
    a = np.array([1,1,-1]).reshape((3,1))
    b = np.array([0])
    max_model.layers[2].set_weights([a,b])
    return max_model#, max_input, max_output



def recursive_max(array):
    length = len(array)
    index_counter = 0
    if length == 1:
        newIn1 = layers.Input(batch_shape = (1,1))
        newOut1 = layers.Dense(1,activation=lambda xy: xy)(newIn1)
        new_model = tf.keras.Model(newIn1, newOut1)
        a = np.array([1]).reshape((1,1))
        b = np.array([0])
        new_model.layers[1].set_weights([a,b])
        return new_model
    elif length == 2:
        new_model = create_max_model_rev()
        return new_model
    else:
        first_arr, second_arr = np.array_split(array,2)
        model_f = recursive_max(first_arr)
        model_s = recursive_max(second_arr)
        #model_f.summary()
        #model_s.summary()
        x = layers.Concatenate()([model_f.output, model_s.output])
        x = layers.Dense(3, activation="relu")(x)
        larger_max = layers.Dense(1)(x)
        model_max = tf.keras.Model([model_f.input, model_s.input], larger_max, name="encoder")
        # first layer
        a = np.array([[1,0,0],[-1,1,-1]])
        b = np.array([0,0,0])
        model_max.layers[-2].set_weights([a,b])

        # second layer
        a = np.array([1,1,-1]).reshape((3,1))
        b = np.array([0])
        model_max.layers[-1].set_weights([a,b])
        index_counter += 1
        return model_max


a_test = tf.convert_to_tensor( np.array([1,2]).reshape(1,2))
b_test = tf.convert_to_tensor( np.array([3,5]).reshape(1,2))

#print(model.layers)
#model = create_max_model_rev()
final_model = recursive_max([1,2,3,4,5])
tensor = tf.reshape( tf.convert_to_tensor( np.array([1,2,3,4,5])),final_model.input)
final_model.summary()
print(final_model.input)
#print(final_model([a_test,b_test]))

